Paper on different types of normalization:

https://genomebiology.biomedcentral.com/articles/10.1186/s13059-021-02568-9#Sec17

Article of different types of feature importance in RandomForestRegressor

https://mljar.com/blog/feature-importance-in-random-forest/



In [20]:
!pip install shap

     -------------------------------------- 447.3/447.3 kB 5.6 MB/s eta 0:00:00


In [25]:
# Load your data
expression_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\simulated_noNoise.txt'
ground_truth_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\bipartite_GRN.csv'

data = pd.read_csv(expression_file, sep="\t")
tg = data.iloc[:,100:]

tg.head()

,100,101,102,103,104,105,106,107,108,109,...,190,191,192,193,194,195,196,197,198,199
0,10.90,5.41,5.76,7.97,8.4,18.8,3.35,10.40,8.35,6.45,...,12.10,5.56,11.0,12.40,5.20,11.80,5.64,4.50,10.20,4.26
1,12.20,7.37,4.51,9.09,11.9,16.7,7.80,10.20,11.10,6.47,...,11.80,6.83,12.0,8.07,4.25,10.00,7.67,4.33,13.70,3.43
2,11.70,2.11,9.77,11.50,14.8,15.9,6.09,10.50,11.20,6.66,...,9.42,7.75,12.6,10.60,4.91,7.89,7.23,3.67,9.49,4.14
3,12.00,6.15,3.39,5.70,15.1,13.2,3.85,11.80,14.30,4.57,...,9.48,4.67,13.7,8.86,6.04,10.60,4.09,5.83,12.50,3.98
4,9.96,7.12,9.37,6.15,15.7,12.5,4.13,7.84,12.30,3.25,...,5.69,9.29,11.9,10.80,6.06,10.40,5.58,2.64,15.50,5.74


In [48]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score

def load_data(expression_file):
    data = pd.read_csv(expression_file, sep="\t")
    tf_expression = data.iloc[:, :100].values  # Assuming TFs are the first 100 columns
    tg_expression = data.iloc[:, 100:].values  # Assuming TGs are the next 100 columns
    return tf_expression, tg_expression

def train_models(tf_expression, tg_expression):
    models = []
    for i in range(tg_expression.shape[1]):
        model = RandomForestRegressor(n_estimators=500,max_features=None)
        model.fit(tf_expression, tg_expression[:, i])
        models.append(model)
    return models

def infer_grn(models, threshold=0.01):
    grn_edges = []
    for i, model in enumerate(models):
        importance = model.feature_importances_
        regulators = np.where(importance > threshold)[0]
        for reg in regulators:
            grn_edges.append((reg, i+100))  # TF ids are 0-99, TG ids are 100-199
    print("here are teh grn edges")    
    print(grn_edges)    
    return grn_edges

def evaluate_grn(predicted_edges, ground_truth_file):
    ground_truth = pd.read_csv(ground_truth_file, header=None)
    ground_truth.head()
    ground_truth_set = set(tuple(x) for x in ground_truth.values)
    predicted_set = set(predicted_edges)
    
    print("here are the metrics for groundtruthset")
    print(ground_truth_set)
    
    print("here are the metrics for predicted_set from my model")
    print(predicted_set)
    #Accuracy (tn calcuation) doesn't seem to be meaningful because there will be more non-regulatory edges then actual edges
    
    tp = len(predicted_set & ground_truth_set)
    fp = len(predicted_set - ground_truth_set)
    fn = len(ground_truth_set - predicted_set)
   
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    print("HERE ARE TP,FP,FN")  
    print(tp,fp,fn)
    return precision, recall

expression_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\simulated_noNoise.txt'
ground_truth_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\bipartite_GRN.csv'

# Load your data and adjust file path as necessary!
tf_expression, tg_expression = load_data(expression_file)

# Train a model for each target gene
models = train_models(tf_expression, tg_expression)

# Infer the gene regulatory network
predicted_grn_edges = infer_grn(models)

# Evaluate the GRN
precision, recall= evaluate_grn(predicted_grn_edges, ground_truth_file)

print(f"Precision: {precision}, Recall: {recall}")


here are teh grn edges
[(24, 100), (25, 100), (39, 100), (44, 100), (45, 100), (52, 100), (54, 100), (57, 100), (62, 100), (69, 100), (73, 100), (74, 100), (77, 100), (80, 100), (83, 100), (89, 100), (90, 100), (93, 100), (97, 100), (98, 100), (22, 101), (50, 101), (90, 101), (94, 101), (99, 101), (10, 102), (17, 102), (26, 102), (29, 102), (30, 102), (37, 102), (38, 102), (39, 102), (56, 102), (74, 102), (83, 102), (14, 103), (23, 103), (31, 103), (32, 103), (43, 103), (45, 103), (52, 103), (54, 103), (58, 103), (64, 103), (75, 103), (95, 103), (9, 104), (10, 104), (24, 104), (29, 104), (39, 104), (45, 104), (57, 104), (58, 104), (59, 104), (72, 104), (83, 104), (85, 104), (3, 105), (4, 105), (11, 105), (16, 105), (18, 105), (19, 105), (20, 105), (27, 105), (28, 105), (33, 105), (36, 105), (38, 105), (43, 105), (53, 105), (63, 105), (66, 105), (71, 105), (72, 105), (73, 105), (85, 105), (87, 105), (89, 105), (91, 105), (96, 105), (98, 105), (8, 106), (9, 106), (12, 106), (15, 106), (1

# Approach 1 - Correlation Pruning

In [23]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor

def correlation_pruning(preliminary_network, expression_data, threshold):
    pruned_network = []
    for tf, tg in preliminary_network:
        correlation, _ = pearsonr(expression_data[:, tf], expression_data[:, tg])  # tg is already referenced from indices 100 onward
        if abs(correlation) > threshold:
            pruned_network.append((tf, tg))
    return pruned_network


expression_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\simulated_noNoise.txt'
ground_truth_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\bipartite_GRN.csv'

# Assume the rest of the functions like train_models and load_data are defined as above
expression_data = pd.read_csv(expression_file, sep="\t", header=None).values

expression[]
# Load your data
tf_expression, tg_expression = load_data(expression_file)

# Train a model for each target gene and construct preliminary network
models = train_models(tf_expression, tg_expression)
preliminary_network = infer_grn(models)  # Modify this function to output (tf, tg) pairs

# Prune the network based on correlation
final_network = correlation_pruning(preliminary_network, expression_data,threshold=0.3)

# Evaluate the final GRN
precision, recall = evaluate_grn(final_network, r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\bipartite_GRN.csv')

print(f"Precision: {precision}, Recall: {recall}")

#THIS DOES NOT WORK REALLY WELL!

Precision: 0.23404255319148937, Recall: 0.6776180698151951


# Approach 2 - Topological graphs and centrality (DOES NOT WORK!)


In [15]:
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.ensemble import RandomForestRegressor

def load_data(expression_file):
    data = pd.read_csv(expression_file, sep="\t", header=None)
    tf_expression = data.iloc[:, :100].values  # Assuming TFs are the first 100 columns
    tg_expression = data.iloc[:, 100:].values  # Assuming TGs are the next 100 columns
    return tf_expression, tg_expression

def train_models(tf_expression, tg_expression):
    models = []
    for i in range(tg_expression.shape[1]):
        model = RandomForestRegressor(n_estimators=100)
        model.fit(tf_expression, tg_expression[:, i])
        models.append(model)
    return models

def infer_grn(models):
    preliminary_network = []
    for tg, model in enumerate(models):
        importance = model.feature_importances_
        # Considering TFs with non-zero importance
        for tf, imp in enumerate(importance):
            if imp > 0:
                preliminary_network.append((tf, tg + 100))  # tg + 100 to adjust index for TGs
    return preliminary_network

def apply_network_topology_analysis(preliminary_network, models):
    G = nx.DiGraph()
    for tf, tg in preliminary_network:
        weight = models[tg - 100].feature_importances_[tf]  # Adjusting index for TGs
        G.add_edge(tf, tg, weight=weight)

    centrality = nx.betweenness_centrality(G, weight='weight')
    refined_network = [(tf, tg) for tf, tg in preliminary_network if centrality[tf] > np.median(list(centrality.values()))]

    return refined_network

def evaluate_grn(predicted_network, ground_truth_file):
    ground_truth = pd.read_csv(ground_truth_file, header=None)
    ground_truth_set = set(map(tuple, ground_truth.values))
    predicted_set = set(predicted_network)
    
    tp = len(predicted_set & ground_truth_set)
    fp = len(predicted_set - ground_truth_set)
    fn = len(ground_truth_set - predicted_set)
    
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    
    return precision, recall

# Load data
expression_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\simulated_noNoise.txt'
ground_truth_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\bipartite_GRN.csv'
tf_expression, tg_expression = load_data(expression_file)

# Train Random Forest models
models = train_models(tf_expression, tg_expression)

# Infer preliminary GRN
preliminary_network = infer_grn(models)

# Refine network using topology analysis
final_network = apply_network_topology_analysis(preliminary_network, models)

# Evaluate the GRN
precision, recall = evaluate_grn(final_network, ground_truth_file)
print(f"Precision: {precision}, Recall: {recall}")


Precision: 0, Recall: 0.0


## Approach 3 - Shapley Feature Importance


In [41]:
import numpy as np
import pandas as pd
import shap
from sklearn.ensemble import RandomForestRegressor

def train_models_with_shap(tf_expression, tg_expression):
    models = []
    shap_values_list = []
    for i in range(tg_expression.shape[1]):
        model = RandomForestRegressor(n_estimators=100)
        model.fit(tf_expression, tg_expression[:, i])

        # Calculate SHAP values
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(tf_expression)
        shap_values_list.append(shap_values)

        models.append(model)
    return models, shap_values_list

def infer_grn_with_shap(models, shap_values_list, threshold):
    grn_edges = []
    for i, (model, shap_values) in enumerate(zip(models, shap_values_list)):
        # Use mean absolute SHAP values as feature importance
        importance = np.abs(shap_values).mean(axis=0)
        regulators = np.where(importance > threshold)[0]
        for reg in regulators:
            grn_edges.append((reg, i+100))  # TF ids are 0-99, TG ids are 100-199
    return grn_edges

# Load your data
expression_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\simulated_noNoise.txt'
ground_truth_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\bipartite_GRN.csv'

tf_expression, tg_expression = load_data(expression_file)

# Train models and calculate SHAP values
models, shap_values_list = train_models_with_shap(tf_expression, tg_expression)

# Infer the gene regulatory network using SHAP values
predicted_grn_edges = infer_grn_with_shap(models, shap_values_list)

# Evaluate the GRN
precision, recall = evaluate_grn(predicted_grn_edges, ground_truth_file)

print(f"Precision: {precision}, Recall: {recall}")


here are the metrics for groundtruthset
{(33, 164), (72, 164), (10, 198), (45, 165), (46, 139), (58, 104), (89, 171), (98, 149), (7, 138), (42, 114), (26, 160), (82, 168), (72, 132), (87, 130), (96, 108), (64, 137), (76, 102), (34, 142), (66, 192), (68, 134), (98, 135), (78, 166), (3, 172), (7, 133), (71, 136), (90, 158), (60, 126), (92, 195), (19, 134), (71, 172), (83, 137), (41, 131), (95, 111), (53, 105), (87, 116), (85, 173), (88, 117), (14, 140), (66, 178), (14, 158), (43, 194), (90, 126), (86, 183), (59, 166), (79, 144), (98, 157), (30, 102), (42, 104), (28, 169), (63, 136), (30, 111), (55, 195), (19, 120), (82, 158), (58, 197), (72, 104), (84, 106), (40, 179), (21, 175), (6, 186), (81, 198), (73, 194), (34, 123), (23, 193), (56, 141), (74, 177), (78, 147), (86, 160), (16, 172), (1, 183), (98, 143), (67, 174), (48, 133), (55, 181), (28, 164), (83, 100), (59, 179), (8, 106), (72, 128), (61, 197), (95, 110), (45, 100), (51, 152), (51, 161), (4, 184), (35, 162), (89, 142), (98, 129)

## Iteration 2 of Shapley Feature Importance - correlation pruning to get rid of the "junk" edges

In [45]:
# Load your data
expression_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\simulated_noNoise.txt'
ground_truth_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\bipartite_GRN.csv'
expression_data = pd.read_csv(expression_file, sep="\t", header=None).values

tf_expression, tg_expression = load_data(expression_file)

# Train models and calculate SHAP values
models, shap_values_list = train_models_with_shap(tf_expression, tg_expression)

# Infer the gene regulatory network using SHAP values
predicted_grn_edges = infer_grn_with_shap(models, shap_values_list,threshold = 0.01)

shap_network_pruned = correlation_pruning(predicted_grn_edges,expression_data,threshold=0.5)

precision, recall = evaluate_grn(shap_network_pruned, ground_truth_file)

print(f"Precision: {precision}, Recall: {recall}")

here are the metrics for groundtruthset
{(33, 164), (72, 164), (10, 198), (45, 165), (46, 139), (58, 104), (89, 171), (98, 149), (7, 138), (42, 114), (26, 160), (82, 168), (72, 132), (87, 130), (96, 108), (64, 137), (76, 102), (34, 142), (66, 192), (68, 134), (98, 135), (78, 166), (3, 172), (7, 133), (71, 136), (90, 158), (60, 126), (92, 195), (19, 134), (71, 172), (83, 137), (41, 131), (95, 111), (53, 105), (87, 116), (85, 173), (88, 117), (14, 140), (66, 178), (14, 158), (43, 194), (90, 126), (86, 183), (59, 166), (79, 144), (98, 157), (30, 102), (42, 104), (28, 169), (63, 136), (30, 111), (55, 195), (19, 120), (82, 158), (58, 197), (72, 104), (84, 106), (40, 179), (21, 175), (6, 186), (81, 198), (73, 194), (34, 123), (23, 193), (56, 141), (74, 177), (78, 147), (86, 160), (16, 172), (1, 183), (98, 143), (67, 174), (48, 133), (55, 181), (28, 164), (83, 100), (59, 179), (8, 106), (72, 128), (61, 197), (95, 110), (45, 100), (51, 152), (51, 161), (4, 184), (35, 162), (89, 142), (98, 129)